# from C++ to python

see https://pybind11.readthedocs.io/en/stable/basics.html

First, get your python environment ready.
We'll use:
```
jupyter-lab pybind11 numpy matplotlib 
```

Pybind11 is a header-only library, so we don't have to compile anything to install it. Just provide the include path to the compiler.
You can test that everything is working with the following command, that retrieves the compilation flag that we will use later. The output depends on your particular software configuration. The first command will show the include path to use pybind11. The second one will tell us the suffix that we have to use for our compiled C++ library. Check that the python version is correctly detected, and the outputs refer to the same python version. If not, your environment is broken. Closing the current terminal and opening a new one may help.

In [1]:
!python -m pybind11 --includes
!python3-config --extension-suffix

-I/u/r/rbertoss/miniconda3/include/python3.10 -I/u/r/rbertoss/Documents/lection_pybind11/venv/lib/python3.10/site-packages/pybind11/include
.cpython-310-x86_64-linux-gnu.so


# Original C++ code

This comes from the exercise about the toy matrix class (lecture 4). We will implement the python interface of that class in python. Note that I modified the `main` function that is present in the repository, renaming it and removing its arguments. Try to use your own implementation.

```c++
#include <iostream>
#include <vector>
#include <fstream>



template <typename T>
class CMatrix {
  public:
    int size;
    std::vector<T> data;
    CMatrix(int N){
      size = N;
      data.resize(N*N);
    }
    CMatrix(){};
    void print_to_file(const std::string& file);
    void read_from_file(const std::string& file);
    CMatrix<T> operator*(const CMatrix<T>& B);
   
};


template <typename T>
void CMatrix<T>::read_from_file(const std::string& file){
  std::ifstream filevar(file);
  if(filevar){
      filevar>>size;
      data.resize(size*size);
  for (int i=0;i<size;++i) {
    for (int j=0;j<size;++j) {
      filevar>>data[i*size+j];
    }
  }
  filevar.close();}
    else{
    std::cout<<"coudn't open the file "<<file<<std::endl;    
  }
};



template <typename T>
void CMatrix<T>::print_to_file(const std::string& file){
  std::ofstream filevar(file);
  filevar<<size<<std::endl;
  for (int i=0;i<size;++i) {
    for (int j=0;j<size;++j) {
      filevar<<data[i*size+j]<<" ";
    }
    filevar<<std::endl;
  }
  filevar.close();
};

template <typename T>
CMatrix<T> CMatrix<T>::operator*(const CMatrix<T>& B){
  if (size != B.size) {
    std::cout<<"The two matrices are not of the same size! The result will be nonsense."<<std::endl;
  }
    CMatrix<T> C(size);
    for (int i=0;i<size;i++){
		for (int j=0;j<size;j++){
			for (int k=0;k<size;k++){
				C.data[i*size + j]+=data[i*size + k]*B.data[k*size + j];
			}
		}
	}
	return C;
};



int old_main (){
  CMatrix<double> A,B;
  A.read_from_file("A.txt");
  B.read_from_file("B.txt");
  auto C=A*B;
  C.print_to_file("C.txt");
  
  return 0;
}

```

# Testing a simple function call without arguments

matrix_cpp.cpp

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");

}

```

now we compile the library (adapt the command to your C++ compiler)

In [2]:
!c++ -O3 -Wall -shared -std=c++14 -fPIC $(python3 -m pybind11 --includes) matrix_cpp.cpp -o matrix_cpp$(python3-config --extension-suffix)

Note: on my system `import matrix_cpp` will look for `matrix_cpp.cpython-310-x86_64-linux-gnu.so`. The name of the file depends on the python version to avoid conflicts.

In [3]:
import matrix_cpp

In [4]:
matrix_cpp?

Type:        module
String form: <module 'matrix_cpp' from '/u/r/rbertoss/Documents/lection_pybind11/AdvancedProgramming_dssc22/python/pybind11/matrix_cpp.cpython-310-x86_64-linux-gnu.so'>
File:        ~/Documents/lection_pybind11/AdvancedProgramming_dssc22/python/pybind11/matrix_cpp.cpython-310-x86_64-linux-gnu.so
Docstring:   This module uses the c++ code to perform simple matrix multiplications


In [5]:
with open('A.txt','w') as f:
    f.write('''3
1 2 3
2 3 4
6 7 8''')
with open('B.txt','w') as f:
    f.write('''3
2 0 0
0 2 0
1 0 2''')

In [6]:
matrix_cpp.test_func?

Docstring:
test_func() -> int

execute old main code
Type:      builtin_function_or_method


In [7]:
matrix_cpp.test_func()

0

In [8]:
with open('C.txt','r') as f:
    print(f.read())

3
5 4 6 
8 6 8 
20 14 16 



# Using a C++ class from python

matrix_cpp.cpp

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");
    using DCMatrix = CMatrix<double>;
    pybind11::class_<DCMatrix > (m, "CMatrix")
            .def(pybind11::init<>())
            .def("read_from_file",&DCMatrix::read_from_file)
            .def("print_to_file",&DCMatrix::print_to_file)
            .def("multiply",&DCMatrix::operator*)
            .def("assign",
                           static_cast< DCMatrix&(DCMatrix::*)( const DCMatrix &) > // select the assignment operator
                            (&DCMatrix::operator=)
                            )
            ;

}

```

Better restart the python kernel after compiling the library...

In [9]:
A=matrix_cpp.CMatrix()
B=matrix_cpp.CMatrix()

In [10]:
A.read_from_file?

Docstring: read_from_file(self: matrix_cpp.CMatrix, arg0: str) -> None
Type:      method


In [11]:
A.read_from_file("A.txt")
B.read_from_file("B.txt")

In [12]:
C=A.multiply(B)

In [13]:
A

In [14]:
B

In [15]:
C

In [16]:
C.print_to_file("C_python.txt")

In [17]:
with open('C_python.txt','r') as f:
    print(f.read())

3
5 4 6 
8 6 8 
20 14 16 



In [18]:
D=matrix_cpp.CMatrix()

In [19]:
D.assign(C)

In [20]:
D.print_to_file("D_python.txt")

In [21]:
with open('D_python.txt','r') as f:
    print(f.read())

3
5 4 6 
8 6 8 
20 14 16 



# Passing data from C++ to python


buffer protocol interface

https://pybind11.readthedocs.io/en/stable/advanced/pycpp/numpy.html

First from C++ to python (easy)

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <stdexcept>
#include <algorithm>

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");
    using DCMatrix = CMatrix<double>;
    pybind11::class_<DCMatrix > (m, "CMatrix")
            .def(pybind11::init<>())
            .def("read_from_file",&DCMatrix::read_from_file)
            .def("print_to_file",&DCMatrix::print_to_file)
            .def("multiply",&DCMatrix::operator*)
            .def("assign",
                           static_cast< DCMatrix&(DCMatrix::*)( const DCMatrix &) >
                            (&DCMatrix::operator=)
                            )
            .def("get_data", [](const DCMatrix & d)
                {
                   //allocate copy of matrix data and pass it to python domain
                   //Python decides when to destroy the object.
                   //The data is completely given to the python world
                   
                   if (d.size <=0 ) {
                      throw std::runtime_error("matrix is empty");
                   }
                   auto * python_data = new double[d.size*d.size];
                   std::copy(d.data.begin(),d.data.end(),python_data);

                   //little routine that is called when
                   // the object is collected by the garbage collector
                   pybind11::capsule free_when_done(python_data, [] (void * pointer) {
                       std::cout << "freeing memory @ " << pointer <<std::endl;
                       delete [] reinterpret_cast<double*>(pointer);
                   });

                   return pybind11::array_t<double>( // array_t is in pybind11/numpy.h
                       {{d.size,d.size}},//shape
                       {d.size*sizeof(double),sizeof(double)}, //stride
                       python_data,
                       free_when_done
                   );

                }
               )
            ;

}
```


In [22]:
data=C.get_data()

In [23]:
data

array([[ 5.,  4.,  6.],
       [ 8.,  6.,  8.],
       [20., 14., 16.]])

# Passing data from python to C++

The direction python -> C++ is difficult because python buffer protocol supports a lot of memory layouts, to make the computation more efficient. Since we don't want to reimplement all possible layout for our simple C++ toy code, we check that the data that python passes is a simple contiguous array. 

```c++
#include "matrix_cpp.hpp"
#include <pybind11/pybind11.h>
#include <pybind11/numpy.h>
#include <stdexcept>
#include <algorithm> 

PYBIND11_MODULE(matrix_cpp, m) {
    m.doc() = "This module uses the c++ code to perform simple matrix multiplications"; // optional module docstring

    m.def("test_func", &old_main, "execute old main code");
    using DCMatrix = CMatrix<double>;
    pybind11::class_<DCMatrix > (m, "CMatrix")
            .def(pybind11::init<>())
            .def("read_from_file",&DCMatrix::read_from_file)
            .def("print_to_file",&DCMatrix::print_to_file)
            .def("multiply",&DCMatrix::operator*)
            .def("assign",
                           static_cast< DCMatrix&(DCMatrix::*)( const DCMatrix &) >
                            (&DCMatrix::operator=)
                            )
            .def("get_data", [](const DCMatrix & d)
                {
                   //allocate copy of matrix data and pass it to python domain
                   //Python decides when to destroy the object.
                   //The data is completely given to the python world

                   if (d.size <=0 ) {
                      throw std::runtime_error("matrix is empty"); 
                   }
                   auto * python_data = new double[d.size*d.size]; 
                   std::copy(d.data.begin(),d.data.end(),python_data);


                   //little routine that is called when
                   // the object is collected by the garbage collector
                   pybind11::capsule free_when_done(python_data, [] (void * pointer) {
                       std::cout << "freeing memory @ " << pointer <<std::endl;
                       delete [] reinterpret_cast<double*>(pointer);
                   });

                   return pybind11::array_t<double>( // array_t is in pybind11/numpy.h
                       {{d.size,d.size}},//shape
                       {d.size*sizeof(double),sizeof(double)}, //stride
                       python_data,
                       free_when_done
                   );

                }
               )
            .def("set_data",[](DCMatrix &d, const pybind11::buffer numpy_matrix)
                {  
                   //get info of python array
                   pybind11::buffer_info info{numpy_matrix.request()};
                   //check that we are dealing with an array of double
                   if (info.format != pybind11::format_descriptor<double>::format()) {
                      throw std::runtime_error("we can accept only a matrix made with C double type");
                   }
                   //sanity check
                   if (info.ndim != 2) {
                       throw std::runtime_error("dimension of array must be 2");
                   }
                   if (info.shape[0] != info.shape[1]){
                       throw std::runtime_error("we implemented only square matrices, sorry");
                   }
                   if (info.shape[0]<=0) {
                      throw std::runtime_error("dimension of the matrix is zero");
                   }
                   // to simplify the logic, implement only contiguous arrays. Check that the array is contiguous
                   ssize_t stride=sizeof(double);
                   for (int i=info.ndim-1;i>=0;--i){
                      if (info.strides[i] != stride) {
                         throw std::runtime_error("sorry, we don't support a non-contiguous matrix");
                      }
                      stride *= info.shape[i];
                   }

                   //all sanity checks are passed, copy the data
                   d = DCMatrix(info.shape[0]); //use assignment to not write other code...
                   std::copy(static_cast<double*>(info.ptr),static_cast<double*>(info.ptr)+info.shape[0]*info.shape[1],d.data.begin());
                   //now c++ is responsable of this data (managed by the smart pointer)
                }
               )
            ;

}

```

In [24]:
import numpy as np

In [25]:
C.set_data(np.array([[1,2,3],[2,3,4],[1,2,3]],dtype=float))

In [26]:
C.get_data()

array([[1., 2., 3.],
       [2., 3., 4.],
       [1., 2., 3.]])

## optional: clean the produced files

(remove the '#')

In [27]:
!#rm A.txt B.txt C.txt C_python.txt D_python.txt matrix_cpp.*.so